In [4]:
import h2o
from h2o.automl import H2OAutoML
import pandas as pd

In [2]:
h2o.init(nthreads=5)

Checking whether there is an H2O instance running at http://localhost:54321 ..... not found.
Attempting to start a local H2O server...
; Java HotSpot(TM) 64-Bit Server VM (build 13.0.1+9, mixed mode, sharing)
  Starting server from D:\Anaconda3\envs\deepcpu\lib\site-packages\h2o\backend\bin\h2o.jar
  Ice root: C:\Users\Vivek\AppData\Local\Temp\tmpkowtte7k
  JVM stdout: C:\Users\Vivek\AppData\Local\Temp\tmpkowtte7k\h2o_Vivek_started_from_python.out
  JVM stderr: C:\Users\Vivek\AppData\Local\Temp\tmpkowtte7k\h2o_Vivek_started_from_python.err
  Server is running at http://127.0.0.1:54321
Connecting to H2O server at http://127.0.0.1:54321 ... successful.


H2O_cluster_uptime:,03 secs
H2O_cluster_timezone:,Asia/Kolkata
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.30.0.1
H2O_cluster_version_age:,1 month and 6 days
H2O_cluster_name:,H2O_from_python_Vivek_c093ke
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,3.973 Gb
H2O_cluster_total_cores:,8
H2O_cluster_allowed_cores:,5
H2O_cluster_status:,"accepting new members, healthy"


In [5]:
train = pd.read_csv("data/train_jqd04QH.csv")
test = pd.read_csv("data/test_KaymcHn.csv")
sub = pd.read_csv("data/sample_submission_sxfcbdx.csv")

In [6]:
trainh2o = h2o.H2OFrame(train)
testh2o = h2o.H2OFrame(test)

Parse progress: |█████████████████████████████████████████████████████████| 100%
Parse progress: |█████████████████████████████████████████████████████████| 100%


In [7]:
x = trainh2o.columns
y = "target"
x.remove(y)
x.remove('enrollee_id')

In [9]:
trainh2o[y] = trainh2o[y].asfactor()

In [ ]:
aml = H2OAutoML(max_models=21, seed=1)
aml.train(x=x, y=y, training_frame=trainh2o)

AutoML progress: |
20:51:01.958: AutoML: XGBoost is not available; skipping it.

████████████████████████████████████████████

In [11]:
lb = aml.leaderboard
lb.head(rows=lb.nrows)

model_id,auc,logloss,aucpr,mean_per_class_error,rmse,mse
StackedEnsemble_BestOfFamily_AutoML_20200510_200309,0.65643,0.372865,0.229991,0.380157,0.332126,0.110308
StackedEnsemble_AllModels_AutoML_20200510_200309,0.655831,0.372706,0.230765,0.383145,0.332024,0.11024
GLM_1_AutoML_20200510_200309,0.653319,0.3735,0.224324,0.38289,0.332305,0.110427
GBM_1_AutoML_20200510_200309,0.643649,0.377299,0.218687,0.391254,0.33384,0.111449
GBM_5_AutoML_20200510_200309,0.640744,0.378019,0.222158,0.386622,0.333332,0.11111
GBM_2_AutoML_20200510_200309,0.63915,0.378509,0.220771,0.387712,0.333788,0.111415
GBM_3_AutoML_20200510_200309,0.632452,0.381617,0.215137,0.390322,0.334973,0.112207
GBM_4_AutoML_20200510_200309,0.624776,0.38786,0.212585,0.400545,0.336773,0.113416
DRF_1_AutoML_20200510_200309,0.623137,0.40868,0.210381,0.399393,0.338811,0.114793


In [ ]:
pred = aml.predict(testh2o)

In [ ]:
sub['target'] = h2o.as_list(pred['p1'])


In [ ]:
sub.to_csv("h2o_ensemble.csv",index=False)